In [ ]:
import xlwings as xw
import requests
import numpy as np
import time

def fugle_stock_crawler(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    json_data = res.json()

    res1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)
    json_data1 = res1.json()

    return {
        "open": json_data["data"]["quote"]["priceOpen"]["price"],
        "high": json_data["data"]["quote"]["priceHigh"]["price"],
        "low": json_data["data"]["quote"]["priceLow"]["price"],
        "close": json_data["data"]["quote"]["trade"]["price"],
        "lastClose": json_data1["data"]["meta"]["priceReference"],
        "name": json_data1["data"]["meta"]["nameZhTw"]
    }

def get_chart_data(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

    result = res.json()

    price_open = []
    price_high = []
    price_low = []
    price_close = []
    unit = []
    volume = []

    time_idx = result["data"]["chart"].keys()

    for time in time_idx:
        data = result["data"]["chart"]
        price_open.append(data[time]["open"])
        price_high.append(data[time]["high"])
        price_low.append(data[time]["low"])
        price_close.append(data[time]["close"])
        unit.append(data[time]["unit"])
        volume.append(data[time]["volume"])

    real_time_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
    stock_data_ary = np.array(real_time_data)
    
    return stock_data_ary.T
# 寫出條件判斷
def check_condition(stock_id, price, condition, limit):
    msg = ""
    if condition == "<":
        if price < limit:
             msg += "{} 的價格低於 {}\n".format(stock_id, limit)
    elif condition == ">":
        if price > limit:
             msg += "{} 的價格高於 {}\n".format(stock_id, limit)
    elif condition == "=":
        if price == limit:
             msg += "{} 的價格等於 {}\n".format(stock_id, limit)
    
    return msg

def line_notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    token = line_token
  
    headers = {
            "Authorization": "Bearer " + token, 
            "Content-Type" : "application/x-www-form-urlencoded"
        }

    payload = {'message': msg}
    r = requests.post(line_url, headers = headers, params = payload)
    return r

# 開啓 dashboard.xlsx
wb = xw.Book(r"dashboard.xlsx")

dashboard = wb.sheets["即時看板"]
watch_list = wb.sheets["觀察清單"]

fugle_token = watch_list.range("L1").value
line_token = watch_list.range("L3").value

while True:
    last_row = watch_list.range("A1").end("down").row
    # 將資料寫入觀察清單
    for i in range(2, last_row+1):
        stock_id = watch_list.range(f"A{i}").value
        stock_id = str(int(stock_id)) if type(stock_id) == float else str(stock_id)
        data = fugle_stock_crawler(stock_id, fugle_token)
        watch_list.range(f"B{i}").value = [data["name"],data["open"],data["high"],data["low"],data["close"],data["lastClose"]]
        
    line_report = "歡樂碼農及時股價看板：\n\n"
    # 截取即時看板上的股票代號
    stock_id = dashboard.range("stock1").value

    # 將被讀取成浮點數的股票代號轉成字串
    if type(stock_id) == float:
        stock_id = str(int(stock_id))    
    
    target_sheet = wb.sheets["走勢資料"]
    # 呼叫函數，將截取到的圖表資料（一個二維陣列）寫入以 A2 為起點的範圍
    target_sheet.range("A2").value = get_chart_data(stock_id, fugle_token)
    # 選擇即時看板的圖表物件
    chart = dashboard.charts["chart1"]
    last_cell = target_sheet.range("E1").end("down")
    # 將圖表物件的資料來源設定成範圍 B2:GN
    chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
    # 將圖表類型設定成 ohlc（開盤-高-低-收盤股價圖）
    chart.chart_type = "stock_ohlc"
    # 將最後一筆收盤價寫入看板工作表
    dashboard.range("price1").value = last_cell.value
    # 讀取觸發 Line 通知的比較條件
    condition = dashboard.range("condition1").value
    limit = dashboard.range("limit1").value
    price = dashboard.range("price1").value
    # 檢查是否符合觸發 Line 通知，產生 Line 訊息的字串
    line_report += check_condition(stock_id, price, condition, limit)

    # 若 Line 訊息的字串與初始化的不符合，意味條件被觸發
    if line_report != "歡樂碼農即時股價看板：\n\n":
        # 發送 Line 訊息
        line_notify(line_report, line_token)

    time.sleep(10)